# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     | Julia Hernández Cárdenas  |
| **Fecha**      | 06/09/2025  |
| **Expediente** | 739208  |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [2]:
ad = pd.read_csv("Advertising.csv")
ad.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


## Sin penalización 

In [3]:
X_todo = ad[['TV', 'radio', 'newspaper']]
X_todo = sm.add_constant(X_todo)
y_todo = ad['sales']

In [4]:
ols = sm.OLS(y_todo, X_todo)
results = ols.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Mon, 08 Sep 2025   Prob (F-statistic):           1.58e-96
Time:                        17:47:07   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9389      0.312      9.422      0.000       2.324       3.554
TV             0.0458      0.001     32.809      0.000       0.043       0.049
radio          0.1885      0.009     21.893      0.000       0.172       0.206
newspaper     -0.0010      0.006     -0.177      0.860      -0.013       0.011
==============================================================================
Omnibus:                       60.414   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.241
Skew:                          -1.327   Prob(JB):                     1.44e-33
Kurtosis:                       6.332   Cond. No.                         454.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:
ad2 = ad[["TV", "radio", "newspaper", "sales"]].dropna().drop_duplicates()

X2 = ad2[["TV", "radio", "newspaper"]]
y2 = ad2["sales"]

In [6]:
scaler = StandardScaler()
X2_scaled = scaler.fit_transform(X2)

In [7]:
def resultado(model, X2):
    model.fit(X2_scaled, y2)
    n = X2_scaled.shape[0]
    p = X2_scaled.shape[1]
    RSS = np.sum((y_pred - y2)**2)
    var = RSS / (n - p - 1)

    b_0 = model.intercept_
    b_i = model.coef_

    X = np.column_stack((np.ones(n), X2_scaled))
    var_beta = np.linalg.inv(X.T @ X) * var
    std_beta = np.sqrt(np.diag(var_beta))

    t_stats = np.array([b_0] + list(b_i)) / std_beta
    p_values = [2 * (1 - stats.t.cdf(np.abs(t), n - p - 1)) for t in t_stats]
    
    print(p_values)

## Ridge

In [8]:
ridge = Ridge(alpha=1.0)
ridge.fit(X2_scaled, y2)

y_pred = ridge.predict(X2_scaled)

print("Coeficientes:", ridge.coef_)
print("Intercepto:", ridge.intercept_)

Coeficientes: [ 3.90021344  2.77691168 -0.0160149 ]
Intercepto: 14.0225


**$R^2$**

In [9]:
r2_ridge = ridge.score(X2_scaled, y2)
print("R^2:", r2_ridge)

R^2: 0.8971891437493419


**p-values**

In [10]:
resultado(ridge,X2)

[0.0, 0.0, 0.0, 0.9002151385581043]


## Lasso

In [11]:
lasso = Lasso(alpha=0.1) 
lasso.fit(X2_scaled, y2)

y_pred_lasso = lasso.predict(X2_scaled)

print("Coeficientes:", lasso.coef_)
print("Intercepto:", lasso.intercept_)

Coeficientes: [3.82360787 2.68932395 0.        ]
Intercepto: 14.0225


**$R^2$**

In [12]:
r2_score(y2, y_pred_lasso)

0.896494232900825

**p-values**

In [13]:
resultado(lasso,X2)

[0.0, 0.0, 0.0, 1.0]
